In [1]:
import torch
from enformer_pytorch import from_pretrained
from enformer_pytorch.finetune import HeadAdapterWrapper

/cellar/users/aklie/opt/miniconda3/envs/enformer-pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
enformer = from_pretrained('EleutherAI/enformer-official-rough', use_tf_gamma = False)

In [7]:
model = HeadAdapterWrapper(
    enformer = enformer,
    num_tracks = 128,
    post_transformer_embed = False   # by default, embeddings are taken from after the final pointwise block w/ conv -> gelu - but if you'd like the embeddings right after the transformer block with a learned layernorm, set this to True
).cuda()
model

HeadAdapterWrapper(
  (enformer): Enformer(
    (stem): Sequential(
      (0): Conv1d(4, 768, kernel_size=(15,), stride=(1,), padding=(7,))
      (1): Residual(
        (fn): Sequential(
          (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): GELU()
          (2): Conv1d(768, 768, kernel_size=(1,), stride=(1,))
        )
      )
      (2): AttentionPool(
        (pool_fn): Rearrange('b d (n p) -> b d n p', p=2)
        (to_attn_logits): Conv2d(768, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
    )
    (conv_tower): Sequential(
      (0): Sequential(
        (0): Sequential(
          (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (1): GELU()
          (2): Conv1d(768, 768, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): Residual(
          (fn): Sequential(
            (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [8]:
seq = torch.randint(0, 5, (1, 196_608 // 2,)).cuda()
target = torch.randn(1, 200, 128).cuda()  # 128 tracks

In [9]:
loss = model(seq, target = target)
loss.backward()

In [10]:
loss

tensor(0.6934, device='cuda:0', grad_fn=<MeanBackward0>)